# 全部采用本地化部署的方式和开源模型来建立的问答能力和本地化知识库集成的能力

定义语言模型

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

/home/xiaodong/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No compiled kernel found.
Compiling kernels : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c -shared -o /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Load kernel : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 6
Using quantization cache
Applying quantization to glm layers


按照langchain标准进行模型封装

In [2]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from llama_index import LLMPredictor

class CustomLLM(LLM):

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response, history = model.chat(tokenizer, prompt, history=[])
        # only return newly generated tokens
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "THUDM/chatglm-6b-int4"}

    @property
    def _llm_type(self) -> str:
        return "custom"
    
llm_predictor = LLMPredictor(llm=CustomLLM())



引入embedding模型

In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))

定义包含embedding模型和语言模型的Service Context

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm_predictor)

构建本地知识库（本地知识的索引库）

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from llama_index import SimpleDirectoryReader
from llama_index import GPTVectorStoreIndex
from llama_index.node_parser import SimpleNodeParser

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)


index = GPTVectorStoreIndex(nodes=nodes, service_context=service_context)
query_engine = index.as_query_engine()

Created a chunk of size 130, which is longer than the specified 100


结合本地知识库和语言模型进行提问

In [6]:
question = """
请用中文回答下述问题：

Q: 你们能配送到天津吗？另外，你们的退货政策是怎样的？
"""
response = query_engine.query(question)
print(response)

The dtype of attention mask (torch.int64) is not bool


A: 支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆。因此，可以配送到天津。关于退货政策，我们会在订单发货后询问并确认客户是否需要退货，如果需要，我们会提供相应的退货政策。


In [7]:
question = """
请用中文回答下述问题：

Q: 请介绍一下ChatGLM？
"""
response = query_engine.query(question)
print(response)

ChatGLM-6B 是一个基于语言模型的人工智能助手，由清华大学 KEG 实验室和智谱 AI 公司开发而成。它可以通过文本对话的形式，为用户提供关于话题的建议、信息或完成指令。ChatGLM-6B 使用的技术包括深度学习、自然语言处理和计算机视觉等。它支持大部分省份的配送，可以查看订单状态。


In [8]:
question = """
请用中文回答下述问题：

Q: 请介绍一下语言模型中的Alpaca。
"""
response = query_engine.query(question)
print(response)

Alpaca是一个基于深度学习的自然语言处理模型，由清华大学 KEG 实验室和智谱AI开发。它被设计用于理解和生成自然语言，可以处理多种任务，如文本分类、机器翻译、问答和文本生成等。

Alpaca 采用了卷积神经网络(CNN)和循环神经网络(RNN)的技术，具有大量的预训练数据和强大的计算能力。它的核心架构包括三个主要部分：1)预训练阶段，使用大量的标记数据训练模型；2)强化学习阶段，使用奖励机制来训练模型；3)训练阶段，使用标记数据来微调模型，使其更好地适应新的任务和数据集。

Alpaca 在自然语言处理领域取得了令人瞩目的成就，已经被应用于多个领域和场景，如智能客服、智能助手、文本分类、机器翻译等。


In [9]:
question = """
请用中文回答下述问题：

Q: 请写一段python代码，展示如何使用Alpaca的模型在本地GPU运行支持语言任务的能力。
"""
response = query_engine.query(question)
print(response)

Alpaca 是一个训练有向网络，可以在本地 GPU 上运行支持语言任务的能力。下面是一段 Python 代码，展示如何使用 Alpaca 模型在本地 GPU 上运行支持语言任务的能力：
```python
import tensorflow as tf
from tensorflow.keras.models import load_model

# 加载 Alpaca 模型
model = load_model('paca_model.h5', tensorflow.keras.layers.Model)

# 将模型保存为 GPU 文件
model.save('paca_model.h5', tensorflow.keras.layers.save_model, 
              'paca_model.pth', tensorflow.keras.layers.save_model)

# 运行模型
print('模型运行成功')
```
这段代码首先使用 `load_model` 函数加载 Alpaca 模型，然后使用 `save_model` 函数将模型保存为 GPU 文件。最后，使用 `运行` 函数运行模型，输出运行结果。


使用新的TextSplitter来对原始内容进行处理，换一个新的index来处理索引，对文章进行总结

In [10]:
from langchain.text_splitter import SpacyTextSplitter

new_text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 2048)
new_parser = SimpleNodeParser(text_splitter=new_text_splitter)
new_documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
new_nodes = new_parser.get_nodes_from_documents(new_documents)



In [11]:
from llama_index import GPTListIndex

list_index = GPTListIndex(nodes=new_nodes, service_context=service_context)

# response = list_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", response_mode="tree_summarize")

query_engine = list_index.as_query_engine(
    response_mode="tree_summarize"
)
response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (1680 > 1024). Running this sequence through the model will result in indexing errors


这段文字是鲁迅先生在《藤野先生》一文中所写的，讲述了作者去仙台医学专门学校的经历以及与藤野先生相处的经历。作者一开始对藤野先生的穿着和习惯感到不适应，但在藤野先生的帮助下，作者学到了很多关于解剖学的知识。作者在仙台的住宿和饮食都受到了特别的待遇，但作者因为不用功和任性，最终未能完成藤野先生布置的作业。最后，藤野先生对作者进行了教育，表达了他的感激之情。

这段文字是鲁迅先生所写的一篇文章，题目为《狂人日记》。文章讲述了作者在中国日本的一所中学就读，由于对中国文化的热爱，作者与同学发生了一些矛盾，也参加了一些政治活动。在解剖实习和局部解剖学课程中，作者遇到了一些难以理解的问题，但通过与老师和同学的交流，作者逐渐获得了知识。在日俄战争期间，作者看到了中国人被枪毙的命运，感受到了战争的残酷。最终，作者决定不再学医学，而是选择学习生物学，与老师和同学告别。整篇文章表达了作者对中国文化的热爱和对战争的厌恶，同时也反映了作者在学习和成长过程中所面临的挑战和思考。

在这段文字中，作者回忆了自己老师的经历，描述了和老师之间复杂关系的演变。虽然多次写信和拍照留念，但和老师之间并没有建立深厚的友谊。尽管老师失踪了，但作者仍然记得他的性格和贡献，他改正的讲义也收藏着，将作为永久的纪念。虽然文章被“正人君子”们所深恶痛疾的体制所困扰，但作者必须继续写作，为那些深恶痛疾的人写下去，以此来表达自己对自由和民主的向往。


换用TreeIndex来试试效果

In [12]:
from llama_index import GPTTreeIndex

# define LLM
tree_index = GPTTreeIndex(nodes=new_nodes, service_context=service_context)
# response = tree_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", mode="summarize")

#query_engine = tree_index.as_query_engine(
#    response_mode="summarize"
#)

query_engine = tree_index.as_query_engine()

response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

这段文字是鲁迅先生写的关于自己的一段经历，描述了他在日本医学专门学校学习的经历和与藤野先生的关系。

藤野先生是一位留学生的导师，他在学生的讲义上增加了许多脱漏的地方，并订正了文法错误，使得学生能够更好地学习和成长。

这段文字也表达了鲁迅先生对于中国传统文化的重视和对于新文化运动的支持。他描述了自己在日本医学专门学校学习的经历，并强调了新文化运动对于中国文化的发展的重要性。

此外，藤野先生还在学生的讲义上增加了许多脱漏的地方，让鲁迅先生感到不满，但藤野先生却对鲁迅先生的信任和尊重感到感激。


尝试Vector Store - pinecone

In [ ]:
'''
# 目前这个不分不工作
import pinecone
from llama_index import StorageContext
from llama_index.vector_stores import PineconeVectorStore
import os

# pinecone_api_key = os.environ.get("PINECONE_API_KEY")
# pinecone_env = os.environ.get("PINECONE_ENV")

pinecone_api_key = os.environ['PINECONE_API_KEY']
pinecone_env = os.environ['PINECONE_ENV']


# init pinecone
pinecone.init(pinecone_api_key, pinecone_env)
pinecone.create_index("xdtest1", dimension=1536, metric="cosine", pod_type="p1") # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

# construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store = PineconeVectorStore(pinecone.Index("xdtest1"))
)

# Load documents and build index
documents_p = SimpleDirectoryReader('./data/mr_fujino').load_data()
index_p = GPTVectorStoreIndex.from_documents(documents_p, service_context=service_context, storage_context=storage_context)

query_engine = index_p.as_query_engine()

response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)
'''

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'www-authenticate': 'API key is missing or invalid for the environment "us-west1-gcp". Check that the correct environment is specified.', 'content-length': '114', 'date': 'Sun, 21 May 2023 13:05:33 GMT', 'server': 'envoy'})
HTTP response body: API key is missing or invalid for the environment "us-west1-gcp". Check that the correct environment is specified.


Faiss Vector Store的使用

In [26]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [27]:
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index import StorageContext, load_index_from_storage

new_text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 2048)
new_parser = SimpleNodeParser(text_splitter=new_text_splitter)
new_documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
new_nodes = new_parser.get_nodes_from_documents(new_documents)

dimention_faiss = 1536
faiss_index = faiss.IndexFlatL2(dimention_faiss)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index_by_faiss = GPTVectorStoreIndex.from_documents(new_documents, storage_context=storage_context)

index_by_faiss.storage_context.persist()


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 7055 tokens
> [build_index_from_nodes] Total embedding token usage: 7055 tokens


In [30]:
# load index from disk
vector_store_loaded = FaissVectorStore.from_persist_dir('./storage')
# 下面这个部分必须要提供persist_dir这个路径，似乎缺省不起作用
storage_context_loaded = StorageContext.from_defaults(vector_store=vector_store_loaded, persist_dir="./storage")
index_by_faiss_loaded = load_index_from_storage(storage_context=storage_context_loaded)

query_engine = index_by_faiss_loaded.as_query_engine()
response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

INFO:root:Loading llama_index.vector_stores.faiss from ./storage/vector_store.json.
Loading llama_index.vector_stores.faiss from ./storage/vector_store.json.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 57 tokens
> [retrieve] Total embedding token usage: 57 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2220 tokens
> [get_response] Total LLM token usage: 2220 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

鲁迅先生回忆了他在仙台的经历，他曾经被藤野先生教导，但最终决定不学医学，而是去学生物学。藤野先生有些悲哀，但他叮嘱鲁迅先生将来照相并及时通信告诉他此后的状况。然而，鲁迅先生多年没有照相，也没有写信，从藤野先生的角度看，他已经杳无消息。最后


Prompt template的支持